In [1]:
import torch
import deepinv
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from pathlib import Path
import time

/home/ratlinghitman/miniconda3/envs/torch/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device


device(type='cuda', index=0)

In [3]:
batch_size = 32
image_size = 32

transform = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.0,), (1.0,))
    ]
)

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root="./data", train=True, transform=transform, download=True),
    batch_size=batch_size,
    shuffle=True,
)

In [4]:
lr = 1e-4
epochs = 100
start_epoch = 0

# pretrained_value = Path("./checkpoints/ddpm_epoch_25.pth")
pretrained_value = None
model = deepinv.models.DiffUNet(in_channels=3, out_channels=1, large_model=False, pretrained=pretrained_value).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
mse = deepinv.loss.MSE()

beta_start = 1e-4
beta_end = 0.02
timesteps = 1000

betas = torch.linspace(beta_start, beta_end, timesteps).to(device)
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

In [5]:
for epoch in range(start_epoch, epochs):
    model.train()
    epoch_loss = 0.0  # Accumulate loss for the epoch
    start_time = time.time()  # Start timing the epoch
    num_batches = len(train_loader)

    for batch_idx, (data, _) in tqdm(enumerate(train_loader), desc="Training", total=num_batches):
        imgs = data.to(device)
        noise = torch.randn_like(imgs)
        t = torch.randint(0, timesteps, (imgs.size(0),), device=device)

        noised_imgs = (
            sqrt_alphas_cumprod[t, None, None, None] * imgs +
            sqrt_one_minus_alphas_cumprod[t, None, None, None] * noise
        )

        optimizer.zero_grad()
        estimated_noise = model(noised_imgs, t, type_t="timestep")
        loss = mse(estimated_noise, noise)
        loss = loss.sum()  # Ensure loss is a scalar for backward()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()  # Accumulate batch loss

    avg_loss = epoch_loss / num_batches  # Average loss for the epoch
    epoch_time = time.time() - start_time  # Calculate epoch time
    print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}, Epoch Time: {epoch_time:.2f} seconds")
    if (epoch + 1) % 5 == 0:
        torch.save(model.state_dict(), f"./checkpoints/cifar10/ddpm_epoch_{epoch+1}.pth")

Training:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [1/100], Average Loss: 21.9345, Epoch Time: 153.52 seconds


Training:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [2/100], Average Loss: 21.8109, Epoch Time: 151.36 seconds


Training:   0%|          | 0/1563 [00:00<?, ?it/s]

...

Epoch [49/100], Average Loss: 21.7601, Epoch Time: 153.82 seconds


Training:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [50/100], Average Loss: 21.7639, Epoch Time: 154.79 seconds


Training:   0%|          | 0/1563 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "trained_ddpm.pth")